# <span style="color:#336699">Web Crop Phenology Metrics Service (WCPMS) Example</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Gabriel Sansigolo<sup><a href="https://orcid.org/0000-0003-0789-5858"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: Nov 22, 2024
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook explains how to use the WCPMS service to extract phenology metrics on regions from <em>Earth Observation Data Cubes</em>.
</div>

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is a supplement to the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Sansigolo, G.; Queiroz, G. R.; Ferreira, K. R.; Adami, M.; Körting, T.<a href="http://www.google.com" target="_blank">A Web Service for Crop Sowing and Harvesting Detection from Earth Observation Data Cubes.
    </div>
</div>

# Introduction
<hr style="border:1px solid #0077b9;">

The Web Crop Phenology Metrics Service (WCPMS) is an web service for calculating phenological metrics based on Phenolopy library and EO Data from the Brazil Data Cube (BDC). It will allow analysts to calculate phenological metrics from data cubes without downloading big EO datasets to their personal computers.

The software run on the server side, so it doesn’t require package installation. Through a simple API, analysts indicate a spatial location or region and the WCPMS will retrieve the phenological metrics associated with spatial locations by calculating it using time series. 

The WCPMS is a web service, and it can be accessed through its API. To facilitate these operations, we have developed an official client—a simple Python library. The WCPMS Client is composed of a group of functions, the main ones are:

- ``get_collections``: returns in list format the unique identifier of each of the data cubes available in the BDC’s SpatioTemporal Asset Catalogs (STAC).

- ``get_description``: returns in dictionary format the information on each of the phenology metrics, such as code, name, description and method. 	

- ``get_phenometrics``: returns in dictionary form all the phenological metrics calculated for the given spatial location.

- ``get_phenometrics_region``: returns in list form dictionary with the phenological metrics calculated for each of the given spatial location based on selected region methodology (all, systematic grid or random grid).

This Jupyter Notebook shows how to use WCPMS in Python.

## Web Service
<hr style="border:1px solid #0077b9;">

The architecture of the proposed web service is made up of two sides: (1) the server-side and (2) the client-side. It will allow analysts to calculate phenological metrics from data cubes with no need to download big EO datasets to their personal computers. 

The web service runs on the server-side, so everything related to it, such as the libraries for calculating phenological metrics, the access to the web service for extracting time series (WTSS), is on the cloud, so it doesn’t require any package installation. 

<div align="center">
    <figcaption><strong>Figure 1</strong> - Architecture of the web service for phenological metrics extraction </figcaption>
    <img src="https://github.com/GSansigolo/wcpms.py/blob/master/docs/sphinx/img/wcpms_architecture.png?raw=true" align="center" width="768"/>
    <br>
</div>

# Python Client API
<hr style="border:1px solid #0077b9;">

For running the examples in this Jupyter Notebook you will need to install the [WCPMS client for Python](https://github.com/brazil-data-cube/wcpms.py). To install it from PyPI using `pip`, use the following command:

In [ ]:
!pip install git+https://github.com/brazil-data-cube/wcpms.py

In order to access the funcionalities of the client API, you should import the `wcpms` package and `os` package, as follows:

In [ ]:
import os
from wcpms import *

After that, you should add the `wcpms` service url:

In [ ]:
wcpms_url = 'https://data.inpe.br/bdc/wcpms' 

The above cell will create an string named `service` that will save the wcpms url for future use.

# Select Data Cube
<hr style="border:1px solid #0077b9;">

Before creating a data cube query, it is important to select a data cubes from the BDC catalog. It is possible to retrive a list with avaliable data cubes with the `get_collections` function:

In [ ]:
colections=get_collections(
    url = wcpms_url
)

colections

# Creating a Data Cube Query
<hr style="border:1px solid #0077b9;">

To make it easier to use, there is a function to create a data cube query called `cube_query`:

In [ ]:
datacube=cube_query(
    collection="S2-16D-2",
    start_date="2022-01-01",
    end_date="2022-12-31",
    freq="16D",
    band="NDVI"
)

# Opening a shapefile to select a region
<hr style="border:1px solid #0077b9;">

To easily open a shapefile you can use the geopandas (gpd) contained in the wcpms library, there is a function called `gpd_shapefile`:

In [ ]:
path_dir = os.path.dirname("")

gpkg = "../../Data/wcpms/selecao_glebas_pr_2022.gpkg"

In [ ]:
polygon = gpd_read_file(os.path.join(path_dir, gpkg))

polygon = polygon[polygon['gid'] == 2115]
polygon

In order to retrieve the phenological metrics associated with a region, it's necessary to define a region methodology (`all`, `systematic` or `random`).

# Retrieving the Phenological Metrics for Region (Systematic)
<hr style="border:1px solid #0077b9;">

In order to retrieve the phenological metrics associated with a region via `systematic` methodology, with that the metrics are calculated using time series `NDVI`, for each of the given spatial location from `January 1st, 2022` to `December 31st, 2022` and distance of 0.04 km, use the `get_phenometrics_region` function:

In [ ]:
pms=get_phenometrics_region(
    url=wcpms_url,
    cube=datacube,
    geom=gdf_to_geojson(polygon["geometry"]),
    method="systematic",
	distance="0.015"
)

In [ ]:
plot_points_region(polygon, pms)

In [ ]:
pms[9]

# Retrieving the Phenological Metrics for Region (Random)
<hr style="border:1px solid #0077b9;">

In order to retrieve the phenological metrics associated with a region via `random` methodology, with that the metrics are calculated using time series `NDVI`, for each of the given spatial location from `January 1st, 2022` to `December 31st, 2022` and random number of point of `30`, use the `get_phenometrics_region` function:

In [ ]:
pms=get_phenometrics_region(
    url=wcpms_url,
    cube=datacube,
    geom=gdf_to_geojson(polygon["geometry"]),
    method="random",
	plot_size=30
)

In [ ]:
plot_points_region(polygon, pms)

In [ ]:
pms[12]

# Retrieve Description
<hr style="border:1px solid #0077b9;">

Once you start using wcpms to extract phenology metrics, the information about each metric becomes very important. To help you with that, we provide a complete description of all the metrics provided by the service. It is possible to plot a table with all the descriptions with the `get_description` function:

In [ ]:
description=get_description(
    url = wcpms_url
)

description

# See also the WCPMS Documentation
<hr style="border:1px solid #0077b9;">

* [WCPMS Documentation](https://wcpms.readthedocs.io/)